In [11]:
import random
from deap import base, creator, tools, algorithms
from graph_tool import Graph, draw
from graph_tool.topology import min_spanning_tree, shortest_distance
import python_codes.files_operators

filename1 = "Germany50"

# 读取网络数据
read_graph, read_pos = python_codes.files_operators.read_files(f"../networks_clusters/{filename1}.net")
print("nodes: ", read_graph.num_vertices(), "links ", read_graph.num_edges())

# 设置DEAP
if not hasattr(creator, "FitnessMin"):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
if not hasattr(creator, "Individual"):
    creator.create("Individual", Graph, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 初始化种群
def initialize_individual(graph):
    mst = min_spanning_tree(graph)
    mst_graph = creator.Individual(directed=False)
    mst_graph.add_vertex(graph.num_vertices())
    for edge, in_mst in zip(graph.edges(), mst):
        if in_mst:
            mst_graph.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in graph.vertex_properties.items():
        mst_graph.vertex_properties[prop_name] = mst_graph.new_vertex_property(prop_map.value_type())
        for v in mst_graph.vertices():
            mst_graph.vertex_properties[prop_name][v] = graph.vertex_properties[prop_name][v]
    for prop_name, prop_map in graph.edge_properties.items():
        mst_graph.edge_properties[prop_name] = mst_graph.new_edge_property(prop_map.value_type())
        for e in mst_graph.edges():
            mst_graph.edge_properties[prop_name][e] = graph.edge_properties[prop_name][graph.edge(e.source(), e.target())]
    return mst_graph

toolbox.register("individual", tools.initIterate, creator.Individual, lambda: initialize_individual(read_graph))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 适应度函数：计算总路径长度
def eval_fitness(individual):
    total_length = 0
    for v in individual.vertices():
        dist_map = shortest_distance(individual, source=v)
        total_length += sum(dist_map.a)
    return total_length,

toolbox.register("evaluate", eval_fitness)

# 交叉操作
def crossover(parent1, parent2):
    size = parent1.num_edges()
    edges1 = list(parent1.edges())
    edges2 = list(parent2.edges())
    child = creator.Individual(directed=False)
    child.add_vertex(parent1.num_vertices())
    for i in range(size // 2):
        edge = edges1[i]
        child.add_edge(edge.source(), edge.target())
    for i in range(size // 2, size):
        edge = edges2[i]
        child.add_edge(edge.source(), edge.target())
    # 复制顶点和边属性
    for prop_name, prop_map in parent1.vertex_properties.items():
        child.vertex_properties[prop_name] = child.new_vertex_property(prop_map.value_type())
        for v in child.vertices():
            child.vertex_properties[prop_name][v] = parent1.vertex_properties[prop_name][v]
    for prop_name, prop_map in parent1.edge_properties.items():
        child.edge_properties[prop_name] = child.new_edge_property(prop_map.value_type())
        for e in child.edges():
            child.edge_properties[prop_name][e] = parent1.edge_properties[prop_name][parent1.edge(e.source(), e.target())]
    del child.fitness.values  # 删除适应度值以便重新评估
    return child

toolbox.register("mate", crossover)

# 变异操作
def mutate(individual):
    if random.random() < 0.1:  # 10%变异概率
        edges = list(individual.edges())
        individual.remove_edge(random.choice(edges))
        available_edges = list(read_graph.edges())
        new_edge = random.choice(available_edges)
        individual.add_edge(new_edge.source(), new_edge.target())
    del individual.fitness.values  # 删除适应度值以便重新评估
    return individual,

toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# 设置遗传算法参数
population = toolbox.population(n=50)
NGEN = 100
CXPB = 0.5
MUTPB = 0.2

# 执行遗传算法
for gen in range(NGEN):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 应用交叉和变异操作
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 评估新个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    # 打印当前代的最佳个体
    fits = [ind.fitness.values[0] for ind in population]
    best_ind = tools.selBest(population, 1)[0]
    print(f"Generation {gen}: Best fitness = {min(fits)}")

best_ind = tools.selBest(population, 1)[0]
print("Best MST nodes: ", best_ind.num_vertices(), "Best MST links: ", best_ind.num_edges())

# 绘制最优解
draw.graph_draw(best_ind, read_pos, vertex_text=best_ind.vertex_properties["number"], edge_color='blue', output_size=(1000, 1000), output=f'./output_GA/DEAP{filename1}_best_mst.pdf')


nodes:  50 links  69


AttributeError: 'Graph' object has no attribute 'fitness'